# Train Networks

- Three-way SoftMax or Multi-BCE classifier of normal, non-vascular MCI, and non-vascular dementia
- `Weights and Biases` sweep is used for hyperparameter search

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# Load some packages
import os
import json
from copy import deepcopy
import gc
import time

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pprint
import wandb

# custom package
from datasets.caueeg_dataset import *
from datasets.caueeg_script import *
from models import *
from train import *

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.11.0+cu113
cuda is available.


-----

## Set the default configuration for building datatset

In [4]:
cfg_data = {}
cfg_data['device'] = device
cfg_data['task'] = 'task1'
cfg_data['dataset_path'] = r'local/dataset/02_Curated_Data_220416/'
cfg_data['file_format'] = 'memmap'  # 'feather', 'memmap'
cfg_data['load_event'] = False
cfg_data['latency'] = 200 * 10      # 10 seconds
cfg_data['crop_length'] = 200 * 10  # 10 seconds
cfg_data['longer_crop_length'] = 200 * 10 * 6 # 60 seconds
cfg_data['crop_multiple'] = 4
cfg_data['minibatch'] = 256
cfg_data['EKG'] = 'O'
cfg_data['photic'] = 'X'
cfg_data['input_norm'] = 'dataset'  # 'datatset', 'datapoint', 'no'
cfg_data['awgn'] = 5e-2
cfg_data['mgn'] = 1e-4
cfg_data['awgn_age'] = 5e-2

In [5]:
cfg_data_temp = deepcopy(cfg_data)
_ = build_dataset_for_train(cfg_data_temp, verbose=True)

in_channels = cfg_data_temp['preprocess_train'](next(iter(_[0])))['signal'].shape[1]
out_dims = len(cfg_data_temp['class_label_to_name'])

del _

transform: Compose(
    EegRandomCrop(crop_length=2000, length_limit=10000000, multiple=4, latency=2000, return_timing=False)
    EegDropChannels(drop_index=[20])
    EegToTensor()
)

----------------------------------------------------------------------------------------------------

transform_longer: Compose(
    EegRandomCrop(crop_length=12000, length_limit=10000000, multiple=4, latency=2000, return_timing=False)
    EegDropChannels(drop_index=[20])
    EegToTensor()
)

----------------------------------------------------------------------------------------------------


task config:
{'class_label_to_name': ['Normal', 'MCI', 'Dementia'],
 'class_name_to_label': {'Dementia': 2, 'MCI': 1, 'Normal': 0},
 'task_description': 'Classification of [Normal], [MCI], and [Dementia] '
                     'symptoms.',
 'task_name': 'CAUEEG-task1 benchmark'}

 ---------------------------------------------------------------------------------------------------- 

train_dataset[0].keys():
dict_keys

In [6]:
pprint.pprint(cfg_data_temp, width=250)

{'EKG': 'O',
 'age_mean': tensor([71.3571]),
 'age_std': tensor([9.8087]),
 'awgn': 0.05,
 'awgn_age': 0.05,
 'class_label_to_name': ['Normal', 'MCI', 'Dementia'],
 'class_name_to_label': {'Dementia': 2, 'MCI': 1, 'Normal': 0},
 'crop_length': 2000,
 'crop_multiple': 4,
 'dataset_name': 'CAUEEG dataset',
 'dataset_path': 'local/dataset/02_Curated_Data_220416/',
 'device': device(type='cuda'),
 'file_format': 'memmap',
 'input_norm': 'dataset',
 'latency': 2000,
 'load_event': False,
 'longer_crop_length': 12000,
 'mgn': 0.0001,
 'minibatch': 256,
 'photic': 'X',
 'preprocess_test': Sequential(
  (0): EegToDevice(device=cuda)
  (1): EegNormalizeAge(mean=tensor([71.3571]),std=tensor([9.8087]),eps=1e-08)
  (2): EegNormalizeMeanStd(mean=tensor([ 0.1397,  0.1296,  0.0475,  0.0021,  0.0343,  0.0937, -0.0329,  0.0039,
           0.0308, -0.0461,  0.0209,  0.0016, -0.0571, -0.0047, -0.1285, -0.0153,
           0.0235, -0.0051,  0.0055, -0.0154]),std=tensor([46.3705, 21.0563, 11.7064, 11.9615, 

-----

## Define Network Models

In [7]:
cfg_common_model = {'in_channels': in_channels, 
                    'out_dims': out_dims}

cfg_model_pool = []

#### 1D Tiny CNN

In [8]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-Tiny-CNN'
cfg_model['generator'] = TinyCNN1D
cfg_model['fc_stages'] = 1
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'fc_stages': 1,
 'final_pool': 'max',
 'generator': <class 'models.simple_cnn_1d.TinyCNN1D'>,
 'in_channels': 20,
 'model': '1D-Tiny-CNN',
 'out_dims': 3,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

TinyCNN1D(
  (conv1): Conv1d(20, 64, kernel_size=(35,), stride=(7,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 64, kernel_size=(7,), stride=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (final_pool): AdaptiveMaxPool1d(output_size=1)
  (fc_stage): Sequential(
    (0): Sequential(
      (0): Linear(in_features=65, out_features=32, bias=False)
      (1): Dro

#### M7 model (fc-age)

In [9]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-Mx'
cfg_model['generator'] = M7
cfg_model['fc_stages'] = 1
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 256
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 256,
 'fc_stages': 1,
 'final_pool': 'max',
 'generator': <class 'models.simple_cnn_1d.M7'>,
 'in_channels': 20,
 'model': '1D-Mx',
 'out_dims': 3,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

M7(
  (conv1): Conv1d(20, 256, kernel_size=(41,), stride=(2,))
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(256, 256, kernel_size=(11,), stride=(1,))
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(256, 512, kernel_size=(11,), stride=(1,))
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=3, 

#### 1D ResNet model (fc-age)

In [10]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-2x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'in_channels': 20,
 'model': '1D-ResNet-2x',
 'out_dims': 3,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn2): BatchNorm1d(6

#### Deeper 1D ResNet model (fc-age)

In [11]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-5x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'in_channels': 20,
 'model': '1D-ResNet-5x',
 'out_dims': 3,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn2): BatchNorm1d(6

#### Shallower 1D ResNet

In [12]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-2x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BasicBlock1D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BasicBlock1D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'in_channels': 20,
 'model': '1D-ResNet-2x',
 'out_dims': 3,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BasicBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn2): BatchNorm

#### Tiny 1D ResNet model

In [13]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-ResNet-1x'
# cfg_model['generator'] = ResNet1D
# cfg_model['block'] = BasicBlock1D
# cfg_model['conv_layers'] = [1, 1, 1, 1]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['LR'] = None
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### Multi-Dilated 1D ResNet model

In [14]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-Multi-Dilated-ResNet-5x'
# cfg_model['generator'] = ResNet1D
# cfg_model['block'] = MultiBottleneckBlock1D
# cfg_model['conv_layers'] = [3, 4, 6, 3]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 32
# cfg_model['LR'] = None
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### 1D ResNeXt-53

In [15]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-5x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '1D-ResNeXt-5x',
 'out_dims': 3,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), groups=32, bias=False)

#### 1D ResNeXt-103

In [16]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-10x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '1D-ResNeXt-10x',
 'out_dims': 3,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), groups=32, bias=Fals

#### 2D ResNet-20 model

In [17]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-2x' # resnet-18 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = BasicBlock2D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.BasicBlock2D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'in_channels': 20,
 'model': '2D-ResNet-2x',
 'out_dims': 3,
 'stft_params': {'complex_mode': 'as_real', 'n_fft': 200},
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv1): Conv2d(20, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock2D(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running

#### 2D ResNet-52 model

In [18]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-5x' # resnet-50 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
    'hop_length': 100,
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.Bottleneck2D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'in_channels': 20,
 'model': '2D-ResNet-5x',
 'out_dims': 3,
 'stft_params': {'complex_mode': 'as_real', 'hop_length': 100, 'n_fft': 200},
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv1): Conv2d(20, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck2D(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

#### 2D ResNeXt-104 model

In [19]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNeXt-10x' # resnet-101 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 8
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.Bottleneck2D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '2D-ResNeXt-10x',
 'out_dims': 3,
 'stft_params': {'complex_mode': 'as_real', 'n_fft': 200},
 'use_age': 'fc',
 'width_per_group': 8}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv1): Conv2d(20, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck2D(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1,

#### CNN-Transformer

In [20]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-CNN-Transformer'
# cfg_model['generator'] = CNNTransformer
# cfg_model['fc_stages'] = 2
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 256
# cfg_model['n_encoders'] = 4
# cfg_model['n_heads'] = 4
# cfg_model['dropout'] = 0.2
# cfg_model['LR'] = None
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')

# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### Summarize the loaded models

In [21]:
for cfg_model in cfg_model_pool:
    pprint.pp(cfg_model, width=150)
    print('\n' + '-' * 100 + '\n')

{'in_channels': 20,
 'out_dims': 3,
 'model': '1D-Tiny-CNN',
 'generator': <class 'models.simple_cnn_1d.TinyCNN1D'>,
 'fc_stages': 1,
 'use_age': 'fc',
 'final_pool': 'max',
 'base_channels': 64,
 'LR': None,
 'activation': 'relu'}

----------------------------------------------------------------------------------------------------

{'in_channels': 20,
 'out_dims': 3,
 'model': '1D-Mx',
 'generator': <class 'models.simple_cnn_1d.M7'>,
 'fc_stages': 1,
 'use_age': 'fc',
 'final_pool': 'max',
 'base_channels': 256,
 'LR': None,
 'activation': 'relu'}

----------------------------------------------------------------------------------------------------

{'in_channels': 20,
 'out_dims': 3,
 'model': '1D-ResNet-2x',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'use_age': 'fc',
 'final_pool': 'max',
 'base_channels': 64,
 'LR': None,
 'activation': 'relu'}

---------------------------

-----

## Default Configurations for Training

In [22]:
# training configurations
cfg_train = {}
cfg_train['iterations'] = (180000 * 32) // cfg_data['minibatch']
cfg_train['num_history'] = 500
cfg_train['lr_decay_timing'] = 0.8
cfg_train['lr_decay_gamma'] = 0.1
cfg_train['weight_decay'] = 1e-2
cfg_train['mixup'] = 0.0 # 0 for no usage
cfg_train['criterion'] = 'cross-entropy' # 'cross-entropy', 'multi-bce'

cfg_train['device'] = device
cfg_train['save_model'] = True
cfg_train['draw_result'] = True
cfg_train['watch_model'] = False

-----

## Train

In [23]:
sweep_data = {}
sweep_data['crop_length'] = {
    'values': [
        200 * 10, # 10 sec
        200 * 15, # 15 sec
        # 200 * 20, # 20 sec
        # 200 * 30, # 30 sec
              ],
}

sweep_data['EKG'] = {
    'values': ['O', 'X'],
}

sweep_data['photic'] = {
    'values': ['O', 'X'],
}

sweep_data['awgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

sweep_data['mgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.1,
}

sweep_data['awgn_age'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

In [24]:
sweep_model = {}
sweep_model['model_index'] = { 
    'values' : [i for i in range(len(cfg_model_pool))] 
}

sweep_model['fc_stages'] = { 
    'distribution' : 'int_uniform',
    'min': 1,
    'max': 4,
}

sweep_model['use_age'] = { 
    'values' : ['fc', 'conv']
}

sweep_model['final_pool'] = { 
    'values' : ['max', 'average']
}

sweep_model['first_dilation'] = { 
    'distribution' : 'int_uniform',
    'min': 1,
    'max': 3,
}

sweep_model['base_stride'] = { 
    'distribution' : 'int_uniform',
    'min': 3,
    'max': 5,
}

sweep_model['base_channels'] = { 
    'values' : [32, 64]
}

sweep_model['dropout'] = {
    'distribution': 'uniform',
    'min': 0.3,
    'max': 0.5
}

# sweep_model['LR'] = {
#     'distribution': 'log_uniform',
#     'min': math.log(5e-5),
#     'max': math.log(2e-3)
# }

sweep_model['activation'] = {
    'values' : ['relu', 'gelu', 'mish']
}

In [25]:
sweep_train = {}
# sweep_train['iterations'] = {
#     'values' : [150000]
# }

# sweep_train['lr_decay_gamma'] = {
#     'distribution' : 'uniform',
#     'min': 0.1,
#     'max': 0.3,
# }

sweep_train['lr_decay_timing'] = {
    'values' : [0.8]
}

sweep_train['weight_decay'] = {
    'distribution' : 'log_uniform',
    'min': math.log(1e-5),
    'max': math.log(1e-1)
}

sweep_train['mixup'] = {
    'values': [0, 0.1, 0.2, 0.3]
}

sweep_train['criterion'] = {
    'values': ['cross-entropy', 'multi-bce']
}

In [26]:
sweep_config = {
    "entity": "ipis-mjkim",
    "name" : "my-sweep",
    "method" : "random",
    "parameters" : 
    {
        **sweep_data,
        **sweep_model,
        **sweep_train,
    }
}

sweep_id = wandb.sweep(sweep_config, project=f"caueeg-{cfg_data['task']}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. weight_decay uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: h6jpbx8k
Sweep URL: https://wandb.ai/ipis-mjkim/caueeg-task1/sweeps/h6jpbx8k


In [27]:
def train():
    # initialize the wandb log
    wandb_run = wandb.init()
    wandb.run.name = wandb.run.id

    with wandb_run:
        # collect some garbages
        time.sleep(10)
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    
        # load default config
        config = {}
        cfg_model = cfg_model_pool[wandb.config.model_index]
        for k, v in {**cfg_data, **cfg_train, **cfg_model}.items():
            if k not in wandb.config:
                config[k] = v

        # to prevent callables from type-conversion to str
        config['history_interval'] = max(config['iterations'] // config['num_history'], 1)
        wandb.config.update(config)
        for k, v in wandb.config.items():
            if k not in config:
                config[k] = v

        # build dataset
        train_loader, val_loader, test_loader, test_loader_longer = build_dataset_for_train(config)
        
        # model in/out configuration
        config['in_channels'] = config['preprocess_train'](next(iter(train_loader)))['signal'].shape[1]
        config['out_dims'] = len(config['class_label_to_name'])
        
        # train the model
        model = train_with_wandb(config, train_loader, val_loader, test_loader, test_loader_longer, 
                                 config['preprocess_train'], config['preprocess_test'])
        
        # release memory
        del model
        del train_loader, val_loader
        del test_loader, test_loader_longer
        del config

In [ ]:
wandb.agent(sweep_id, function=train, count=100)

wandb: Agent Starting Run: fzkcymwg with config:
wandb: 	EKG: O
wandb: 	activation: gelu
wandb: 	awgn: 0.24060928320260652
wandb: 	awgn_age: 0.1516519768150765
wandb: 	base_channels: 64
wandb: 	base_stride: 3
wandb: 	criterion: multi-bce
wandb: 	crop_length: 3000
wandb: 	dropout: 0.43802236099576086
wandb: 	fc_stages: 4
wandb: 	final_pool: average
wandb: 	first_dilation: 3
wandb: 	lr_decay_timing: 0.8
wandb: 	mgn: 0.02954655932233188
wandb: 	mixup: 0.2
wandb: 	model_index: 7
wandb: 	photic: O
wandb: 	use_age: fc
wandb: 	weight_decay: 0.007794167060441397
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ipis-mjkim (use `wandb login --relogin` to force relogin)


************************************************************************************************************************
******************************                 2D-ResNet-2x train starts                  ******************************
************************************************************************************************************************
